<a href="https://colab.research.google.com/github/kush9405/100-days-of-deep-learning/blob/main/text_summarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub
daily_mail=kagglehub.dataset_download('evilspirit05/daily-mail-summarization-dataset')

In [3]:
daily_mail

'/root/.cache/kagglehub/datasets/evilspirit05/daily-mail-summarization-dataset/versions/1'

In [4]:
import pandas
df=pandas.read_csv('/root/.cache/kagglehub/datasets/evilspirit05/daily-mail-summarization-dataset/versions/1/article_highlights.csv')

In [5]:
df.head()

,url,article,highlights
0,https://www.dailymail.co.uk/tvshowbiz/article-...,Beyoncé showcases her incredible figure in plu...,Beyoncé has shown off her flawless beauty in a...
1,https://www.dailymail.co.uk/tvshowbiz/article-...,Radio 1 listeners in shock as sex noises are p...,BBC Radio 1 listeners were left choking on the...
2,https://www.dailymail.co.uk/tvshowbiz/article-...,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Dan Edgar and Ella Rae Wise put on a loved-up ...
3,https://www.dailymail.co.uk/tvshowbiz/article-...,Bradley Cooper recalls 'crazy' pitch meeting a...,Bradley Cooper discussed the 'crazy' experienc...
4,https://www.dailymail.co.uk/tvshowbiz/article-...,Margaret Qualley and Beanie Feldstein stun in ...,Margaret Qualley and Beanie Feldstein were dre...


In [6]:
df.shape

(8176, 3)

In [7]:
df.isnull().sum()

,0
url,0
article,11
highlights,3


In [8]:
import pandas as pd
# df['url'] = df['url'].astype(str)  # Assuming your column is column 0
# X = df.iloc[:1, 0:1].values.tolist()
X = [item[0] for item in df.iloc[:, 0:1].values.tolist()]
y=df.iloc[:,-1]

In [9]:
X

['https://www.dailymail.co.uk/tvshowbiz/article-13108775/Beyonce-figure-plunging-dress-leg-slit-haircare-brand.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13108329/radio-1-sex-noise-prank-greg-james-breakfast-show.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13108013/TOWIEs-Dan-Edgar-33-Ella-Rae-Wise-23-loved-display-trip-romance-blossoms-pair-trip-Bali.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13107689/Bradley-Cooper-recalls-crazy-meeting-Beyonce-Jay-Zs-home.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13107045/Margaret-Qualley-Beanie-Feldstein-Drive-Away-Dolls-premiere-NYC.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13106099/Selena-Gomez-puts-busty-display-LBD-teamed-chic-head-scarf-heads-dinner-Paris.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13063695/bush-theatre-review-london-patrick-marmion.html',
 'https://www.dailymail.co.uk/tvshowbiz/article-13108775/Beyonce-figure-plunging-dress-leg-slit-haircare-brand.html',


In [10]:
# """## Example Usage with a List of URLs"""
# i=0
# from tabulate import tabulate
# # List of URLs
# # urls = [
# #     'https://www.dailymail.co.uk/tvshowbiz/article-13108775/Beyonce-figure-plunging-dress-leg-slit-haircare-brand.html',
# #     'https://www.dailymail.co.uk/tvshowbiz/article-13108329/radio-1-sex-noise-prank-greg-james-breakfast-show.html'
# # ]
# urls=X
# articles = []
# summaries = []
# success = []  # Track scraping success

# for url in urls:
#     print(f"Record No: {i}")
#     i+=1
#     print(f"Processing URL: {url}")
#     article_text = scrape_article(url)

#     if article_text:
#         summary = extractive_summarization_bert(article_text, num_sentences=3)
#         articles.append(article_text)
#         summaries.append(summary)
#         success.append(True)
#         # print(f"Summary for {url}:\n{summary}\n")
#     else:
#         articles.append(None)
#         summaries.append(None)
#         success.append(False)
#         print(f"Could not scrape article from {url}.\n")


# # Define maximum column widths
# max_url_width = 30
# max_article_width = 30
# max_summary_width = 70
# max_success_width = 10


# # Create DataFrame for Tabular Output
# data = {'URL': urls, 'Article': articles, 'Summary': summaries, 'Success': success}

# # Truncate the text data before creating the DataFrame
# truncated_data = {
#     'URL': [wrap_text(url, max_url_width,5) for url in data['URL']],
#     'Article': [wrap_text(article, max_article_width,2) if article else "" for article in data['Article']],
#     'Summary': [wrap_text(summary, max_summary_width,5) if summary else "" for summary in data['Summary']],
#     'Success': data['Success']  # No wrapping needed for boolean values
# }

# df_results = pd.DataFrame(truncated_data)


# # Define column formats
# column_widths = [max_url_width, max_article_width, max_summary_width, max_success_width]  # Match the defined widths
# headers = df_results.columns

# # Convert the DataFrame to a list of lists
# table_data = [df_results.iloc[i].tolist() for i in range(len(df_results))]

# # Format the table using tabulate
# table = tabulate(table_data, headers=headers, tablefmt="grid", colalign=("left", "left", "left", "center"))  # Other table formats available

# # Print the table
# print("\n--- Tabular Results ---")
# print(table)

In [11]:
import requests
from bs4 import BeautifulSoup
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re  # For robust sentence splitting
import pandas as pd  # Added import for pandas
from tabulate import tabulate # Added import for tabulate
import textwrap

In [12]:
def wrap_text(text, max_width, max_lines, truncate_string="..."):
    """Wraps text to fit within a maximum width and limits the number of lines.

    Args:
        text: The text to wrap.
        max_width: The maximum width of each line.
        max_lines: The maximum number of lines to allow.
        truncate_string: The string to use if the text is truncated.

    Returns:
        The wrapped text, truncated to the specified number of lines.
    """
    if text is None:
        return ""

    wrapped_text = textwrap.fill(text, width=max_width)
    lines = wrapped_text.splitlines()  # Split into individual lines

    if len(lines) > max_lines:
        # Truncate if there are more lines than allowed
        wrapped_text = "\n".join(lines[:max_lines]) + truncate_string #Keep specified lines and then append
        #print("TEXT IS TOO LONG") #DEBUG
    else:
        wrapped_text = "\n".join(lines)  #Just combine if it's short
        #print("Text is small")

    return wrapped_text


In [13]:
def scrape_article(url, max_words=500):
    """
    Scrapes the textual content from a given URL and limits it to a maximum number of words.

    Args:
        url: The URL of the article to scrape.
        max_words: The maximum number of words to extract.

    Returns:
        The textual content of the article, truncated to max_words, or None if scraping fails.
    """
    try:
        print(f"Attempting to scrape: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # **IMPORTANT:** Adapt this section to match the structure of the website you're scraping! **
        # Inspect the HTML source of the page to find the correct tags/classes.

        # Example (adapt to your target website):
        # paragraphs = soup.find_all('p', class_='article-text')  # <p> tags with class "article-text"

        # Generic fallback (prone to grabbing unnecessary content):
        article = soup.find('body')  # Trying to get as much text as possible, but this is not precise
        paragraphs = article.find_all('p')

        if paragraphs:
            text = '\n'.join([p.get_text() for p in paragraphs])  # Corrected: Call get_text()

            # Truncate the text to a maximum of max_words
            words = text.split()
            if len(words) > max_words:
                text = ' '.join(words[:max_words])
                print(f"Article truncated to {max_words} words.")  # Added message

            return text
        else:
            return None  # No article content found.

    except requests.exceptions.RequestException as e:
        print(f"Error during request for {url}: {e}")
        return None
    except Exception as e:
        print(f"Error during scraping {url}: {e}")
        return None



In [14]:
def extractive_summarization_bert(article_text, num_sentences=3):
    """
    Performs extractive summarization using BERT.

    Args:
        article_text: The text of the article.
        num_sentences: The number of sentences to include in the summary.

    Returns:
        The extractive summary.
    """

    # 1. Split the article into sentences (more robust splitting)
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_text)

    # 3. Generate sentence embeddings
    sentence_embeddings = []
    for sentence in sentences:
        if not sentence.strip():  # Skip empty sentences
            continue

        # Tokenize and convert to input IDs
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)  # PyTorch tensors

        # 4. Move Data to GPU
        inputs = {k: v.to(device) for k, v in inputs.items()} # Move all input tensors to the device

        # Get BERT embeddings
        with torch.no_grad():  # Disable gradient calculation for inference
            outputs = model(**inputs)
            # Use the CLS token embedding as the sentence embedding
            sentence_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [batch_size, hidden_size]

        sentence_embeddings.append(sentence_embedding)

    # 4. Calculate sentence scores (similarity to the article)
    if not sentence_embeddings:
        return "Could not generate sentence embeddings."  # Handle case with no valid sentences

    article_embedding = np.mean(np.concatenate(sentence_embeddings, axis=0), axis=0, keepdims=True)  # Average of all sentence embeddings
    sentence_scores = []
    for embedding in sentence_embeddings:
        similarity = cosine_similarity(embedding, article_embedding)[0][0]
        sentence_scores.append(similarity)

    # 5. Select top N sentences
    ranked_sentences = sorted(((score, sentence) for score, sentence in zip(sentence_scores, sentences)), reverse=True)
    summary_sentences = [sentence for score, sentence in ranked_sentences[:num_sentences]]

    # 6. Combine the sentences into a summary
    summary = ". ".join(summary_sentences) + "."  # Re-add periods
    return summary


In [15]:

"""## Example Usage with a List of URLs"""

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")  # Debugging print

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Move Model to GPU
model.to(device)
model.eval() # Set model to evaluation mode

# List of URLs
# urls = [
#     'https://www.dailymail.co.uk/tvshowbiz/article-13108775/Beyonce-figure-plunging-dress-leg-slit-haircare-brand.html',
#     'https://www.dailymail.co.uk/tvshowbiz/article-13108329/radio-1-sex-noise-prank-greg-james-breakfast-show.html'
# ]
urls=X

articles = []
summaries = []
success = []  # Track scraping success

i=0
for url in urls:
    print(f"Record no: {i}")
    i=i+1
    print(f"Processing URL: {url}")
    article_text = scrape_article(url)

    if article_text:
        summary = extractive_summarization_bert(article_text, num_sentences=3)
        articles.append(article_text)
        summaries.append(summary)
        success.append(True)
        # print(f"Summary for {url}:\n{summary}\n")
    else:
        articles.append(None)
        summaries.append(None)
        success.append(False)
        print(f"Could not scrape article from {url}.\n")

# Define maximum column widths and lines
max_url_width = 30
max_url_lines = 2
max_article_width = 30
max_article_lines = 5
max_summary_width = 70
max_summary_lines = 3


# Create DataFrame for Tabular Output
data = {'URL': urls, 'Article': articles, 'Summary': summaries, 'Success': success}

# Truncate the text data before creating the DataFrame
truncated_data = {
    'URL': [wrap_text(url, max_url_width, max_url_lines) for url in data['URL']],
    'Article': [wrap_text(article, max_article_width, max_article_lines) if article else "" for article in data['Article']],
    'Summary': [wrap_text(summary, max_summary_width, max_summary_lines) if summary else "" for summary in data['Summary']],
    'Success': data['Success']  # No wrapping needed for boolean values
}

df_results = pd.DataFrame(truncated_data)

# Define column formats
column_widths = [max_url_width, max_article_width, max_summary_width, 10]  # Match the defined widths
headers = df_results.columns

# Convert the DataFrame to a list of lists
table_data = [df_results.iloc[i].tolist() for i in range(len(df_results))]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid", colalign=("left", "left", "left", "center"))  # Other table formats available

# Print the table
print("\n--- Tabular Results ---")
print(table)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
|                                   | February 2024 19 View comments... |                                                                           |           |
+-----------------------------------+-----------------------------------+---------------------------------------------------------------------------+-----------+
| https://www.dailymail.co.uk/tv    | By ASHLEIGH GRAY FOR              | Gomez draped a flowing black scarf over her head, slightly covered her    |     1     |
| showbiz/article-...               | DAILYMAIL.COM Published: 16:24    | long, dark locks.. She wore a green tank top and put on a sexy display    |           |
|                                   | EDT, 20 February 2024 |           | as she bared her legs, which were crossed in the image.. The Texas-...    |           |
|                                   | Updated: 16:43 EDT, 20            |                                                                  